In [3]:
import argparse
import math
import socket
import importlib
import os
import pickle

from datetime import datetime
#import h5pyprovider
import numpy as np
import tensorflow as tf
from keras import backend as K
import matplotlib.pylab as plt
# os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES']="1"

import sys
BASE_DIR = os.path.abspath('')
print(BASE_DIR)
sys.path.append(os.path.join(BASE_DIR, 'models'))
sys.path.append(BASE_DIR) # model
sys.path.append(os.path.join(BASE_DIR, 'tf_utils'))
import provider
import tf_util

/home/az/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/az/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


/home/az/Documents/D-FCN


In [2]:
# Argparses
EPOCH_CNT = 0

BATCH_SIZE = 6
NUM_POINT = 2048
MAX_EPOCH = 200
BASE_LEARNING_RATE = .01
# GPU_INDEX = 7
GPU_INDEX = 1
MOMENTUM = 0.9
OPTIMIZER = 'adam'
DECAY_STEP = 20000
DECAY_RATE = 0.5

In [3]:
MODEL = importlib.import_module('DFCN_pointnet2_group2') # import network module
MODEL_FILE = os.path.join(BASE_DIR, 'DFCN_pointnet2_group2'+'.py')
LOG_DIR = 'log_wen_v16_sample8192_group2_lw14_F1_noheight'
if not os.path.exists(LOG_DIR): os.mkdir(LOG_DIR)
os.system('cp models/DFCN_pointnet2.py %s' % (LOG_DIR)) # bkp of model def
os.system('cp tf_utils/DFCN_util_xy2.py %s' % (LOG_DIR)) # bkp of model def
# os.system('cp train_pointsift_lx_npsplit_V16.ipynb %s' % (LOG_DIR)) # bkp of train procedure
LOG_FOUT = open(os.path.join(LOG_DIR, 'log_train.txt'), 'w')
# LOG_FOUT.write(str(FLAGS)+'\n')

BN_INIT_DECAY = 0.5
BN_DECAY_DECAY_RATE = 0.5
BN_DECAY_DECAY_STEP = float(DECAY_STEP)
BN_DECAY_CLIP = 0.99

HOSTNAME = socket.gethostname()

NUM_CLASSES = 9

BASE_DIR /home/az/Documents/D-FCN/tf_utils


In [4]:
def log_string(out_str):
    LOG_FOUT.write(out_str+'\n')
    LOG_FOUT.flush()
    print(out_str)

def get_learning_rate(batch):
    learning_rate = tf.train.exponential_decay(
                        BASE_LEARNING_RATE,  # Base learning rate.
                        batch * BATCH_SIZE,  # Current index into the dataset.
                        DECAY_STEP,          # Decay step.
                        DECAY_RATE,          # Decay rate.
                        staircase=True)
    learning_rate = tf.maximum(learning_rate, 0.00001) # CLIP THE LEARNING RATE!!
    return learning_rate        

def get_bn_decay(batch):
    bn_momentum = tf.train.exponential_decay(
                      BN_INIT_DECAY,
                      batch*BATCH_SIZE,
                      BN_DECAY_DECAY_STEP,
                      BN_DECAY_DECAY_RATE,
                      staircase=True)
    bn_decay = tf.minimum(BN_DECAY_CLIP, 1 - bn_momentum)
    return bn_decay

def Acc_from_confusions(confusions):
    
    TP = np.diagonal(confusions, axis1=-2, axis2=-1)
    TP_plus_FN = np.sum(confusions, axis=-1)
    TP_plus_FP = np.sum(confusions, axis=-2)
    
    mAcc = np.sum(TP)/np.sum(confusions)
    
    precision = TP / (TP_plus_FP + 1e-6)
    recall = TP / (TP_plus_FN+ 1e-6)
    fscore = 2*(precision * recall)/(precision + recall + 1e-6)
    
    ave_F1 = np.mean(fscore)
    
    s = 'Overall accuracy：{:5.2f}  Average F1 score：{:5.2f} \n'.format(100 * mAcc, 100 * ave_F1)
    s += log_acc(precision)
    s += log_acc(recall)
    s += log_acc(fscore)
    
    log_string(s)
    
    return mAcc, ave_F1
    

def log_acc(acc_list):
    s = ""
    for acc in acc_list:
        s += '{:5.2f} '.format(100 * acc)
    s += '\n'
    return s

def pc_normalize(pc):
    l = pc.shape[0]
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc
    
def drawPlot(x,y,name):
    plt.rcParams['savefig.dpi'] = 300 
    plt.plot(np.arange(0,len(x)),x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.plot(np.arange(0,len(y)),y,'r-',alpha=1,label='Test max: '+str(round(max(y),3))+', min: '+str(round(min(y),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(LOG_DIR+"/"+name+".png",bbox_inches='tight')
    plt.show()
    
def drawF1Plot(x,name):
    plt.rcParams['savefig.dpi'] = 300 
    plt.plot(np.arange(0,len(x)),x,'k-',alpha=1,label='Train max: '+str(round(max(x),3))+', min: '+str(round(min(x),3)))
    plt.legend()
    plt.xlabel('epoch',fontsize=9)
    plt.ylabel(name+' value',fontsize=9)
    plt.savefig(LOG_DIR+"/"+name+".png",bbox_inches='tight')
    plt.show()
    
    
def placeholder_inputs(batch_size, num_point):
    pointclouds_pl = tf.placeholder(tf.float32, shape=(None, None, 3))
    labels_pl = tf.placeholder(tf.int32, shape=(None, None))
    smpws_pl = tf.placeholder(tf.float32, shape=(None, None))
    return pointclouds_pl, labels_pl, smpws_pl

def pc_normalize_min_max(data):
    mindata = np.min(data[:,:3], axis=0)
    maxdata = np.max(data[:,:3], axis=0)
    return 2*(data[:,:3] - mindata)/(maxdata - mindata)

def pc_normalize_min(data):
    mindata = np.min(data[:,:3], axis=0)
    # MS: Function now returns mindata here
    return (data[:,:3] - mindata), mindata

def get_batch(dataset, index, npoints = NUM_POINT):
  
    if(dataset =='train'):
        cub_l = 30.0
        cub_w = 30.0
        cub_h = 100.0
        point_set =  trainSet[:,:3] - np.min(trainSet[:,:3], axis=0)
        semantic_seg = trainSet[:,4].astype(np.int32)
        coordmax = np.max(point_set,axis=0)
        coordmin = np.min(point_set,axis=0)
        smpmin = np.maximum(coordmax-[cub_l,cub_w,cub_h], coordmin)
        smpmin[2] = coordmin[2]
        smpsz = np.minimum(coordmax-smpmin,[cub_l,cub_w,cub_h])
        smpsz[2] = coordmax[2]-coordmin[2]
        isvalid = False
        for i in range(10):
            curcenter = point_set[np.random.choice(len(semantic_seg),1)[0],:]
            curmin = curcenter-[cub_l/2,cub_w/2,cub_h/2]
            curmax = curcenter+[cub_l/2,cub_w/2,cub_h/2]
            curmin[2] = coordmin[2]
            curmax[2] = coordmax[2]
            curchoice = np.sum((point_set>=(curmin-0.0))*(point_set<(curmax+0.0)),axis=1)==3
            cur_point_set = point_set[curchoice,:]
            cur_semantic_seg = semantic_seg[curchoice]
            cur_feat_set = trainFeats[curchoice,:]
    #         if len(cur_semantic_seg)<npoints:
            if len(cur_semantic_seg)==0:
                continue
            mask = np.sum((cur_point_set>=(curmin-0.0))*(cur_point_set<(curmax+0.0)),axis=1)==3
            vidx = np.ceil((cur_point_set[mask,:2]-curmin[:2])/(curmax[:2]-curmin[:2])*[31.0,31.0])
            vidx = np.unique(vidx[:,0]*31.0+vidx[:,1])
            isvalid = np.sum(cur_semantic_seg>-1)/1.0/len(cur_semantic_seg)>=0.7 and len(vidx)/31.0/31.0>=0.3
#             print('isvalid', isvalid,len(vidx)/31.0/31.0,np.sum(cur_semantic_seg>-1),len(cur_semantic_seg))
            if isvalid:
                break
        choice = np.random.choice(len(cur_semantic_seg), npoints, replace=True) 
        point_set = cur_point_set[choice,:]
        feature_set = cur_feat_set[choice,:]
        semantic_seg = cur_semantic_seg[choice]
        mask = mask[choice]
        sample_weight = labelweights[semantic_seg]
        sample_weight *= mask
        return point_set, semantic_seg, sample_weight,feature_set
    
    if(dataset =='test'):
        ##original 30/30/100
        #100 - 1000 = 500
        #300000
        #1000000
        cub_l = 30000000.0
        cub_w = 30000000.0
        cub_h = 100000000.0
        #all rows, 0&1&2 columns = xyz 
        point_set =  testSet[:,:3] - np.min(testSet[:,:3], axis=0)#minimum in column?/ min of x,y,z
        #5th column = classification, used for batch label for corretness
        semantic_seg = testSet[:,4].astype(np.int32)
        coordmax = np.max(point_set,axis=0)
        coordmin = np.min(point_set,axis=0)
        smpmin = np.maximum(coordmax-[cub_l,cub_w,cub_h], coordmin) #cub covers much more pts in our dataset
        smpmin[2] = coordmin[2]#y?
        smpsz = np.minimum(coordmax-smpmin,[cub_l,cub_w,cub_h])
        smpsz[2] = coordmax[2]-coordmin[2]
        isvalid = False
        for i in range(10):
            curcenter = point_set[np.random.choice(len(semantic_seg),1)[0],:]
            curmin = curcenter-[cub_l/2,cub_w/2,cub_h/2]
            curmax = curcenter+[cub_l/2,cub_w/2,cub_h/2] 
            curmin[2] = coordmin[2]
            curmax[2] = coordmax[2]
            #len = test_datalen, all false
            #curchoice need to > 1
            #https://stackoverflow.com/questions/40857930/how-does-numpy-sum-with-axis-work
            # #point>min * #point<max -> sum of each coordinate
            curchoice = np.sum((point_set>=(curmin-0.0))*(point_set<(curmax+0.0)),axis=1)==3
            #list filtered after T/F
            #empty
            cur_point_set = point_set[curchoice,:]#!
            cur_semantic_seg = semantic_seg[curchoice]#
            cur_feat_set = testFeats[curchoice,:]
    #         if len(cur_semantic_seg)<npoints:
            if len(cur_semantic_seg)==0:
                continue
            mask = np.sum((cur_point_set>=(curmin-0.0))*(cur_point_set<(curmax+0.0)),axis=1)==3
            vidx = np.ceil((cur_point_set[mask,:2]-curmin[:2])/(curmax[:2]-curmin[:2])*[31.0,31.0])
            vidx = np.unique(vidx[:,0]*31.0+vidx[:,1])
            isvalid = np.sum(cur_semantic_seg>-1)/1.0/len(cur_semantic_seg)>=0.7 and len(vidx)/31.0/31.0>=0.3
#             print('isvalid', isvalid,len(vidx)/31.0/31.0,np.sum(cur_semantic_seg>-1),len(cur_semantic_seg))
            if isvalid:
                break
            
        #(range, output size, replacement) ... got npoints repeates
        #2039808/2048=99.6
        #problem @cur_point_set
        #cur_semantic_seg = [0], len = 1 -> all repeats
        choice = np.random.choice(len(cur_semantic_seg), npoints, replace=True) #npoints= 2048 = repeat
        #
        point_set = cur_point_set[choice,:]#! repeat #choose x,y,z, n rows 
        #
        feature_set = cur_feat_set[choice,:]
        #for batch label
        semantic_seg = cur_semantic_seg[choice]
        mask = mask[choice]
        sample_weight = labelweights[semantic_seg]
        #
        sample_weight *= mask
        return point_set, semantic_seg, sample_weight,feature_set,cur_semantic_seg#cur_semantic_seg

def get_batch_wdp(dataset, batch_idx):
    bsize = BATCH_SIZE
    batch_data = np.zeros((bsize, NUM_POINT, 3))
    batch_feats = np.zeros((bsize, NUM_POINT, 1))
    batch_label = np.zeros((bsize, NUM_POINT), dtype=np.int32)
    batch_smpw = np.zeros((bsize, NUM_POINT), dtype=np.float32)
    for i in range(bsize):
        ps,seg,smpw,feat = get_batch('train',index=0)
        ps = pc_normalize_min(ps)
        batch_data[i,...] = ps
        batch_label[i,:] = seg
        batch_smpw[i,:] = smpw
        batch_feats[i,:] = feat

        dropout_ratio = np.random.random()*0.875 # 0-0.875
        drop_idx = np.where(np.random.random((ps.shape[0]))<=dropout_ratio)[0]
        batch_data[i,drop_idx,:] = batch_data[i,0,:]
        batch_label[i,drop_idx] = batch_label[i,0]
        batch_smpw[i,drop_idx] *= 0
        
    return batch_data, batch_label, batch_smpw, batch_feats  

def get_batch_wdp_test(dataset, batch_idx):
    bsize = BATCH_SIZE
    batch_data = np.zeros((bsize, NUM_POINT, 3))
    ps_xyz_min = np.zeros((bsize, 1, 3))
    ps_out = np.zeros((bsize, NUM_POINT, 3))

    batch_feats = np.zeros((bsize, NUM_POINT, 1))
    batch_label = np.zeros((bsize, NUM_POINT), dtype=np.int32)
    batch_smpw = np.zeros((bsize, NUM_POINT), dtype=np.float32)
    for i in range(bsize):#6
        #point_set, semantic_seg, sample_weight,feature_set,cur_semantic_seg#cur_semantic_seg
        ps,seg,smpw,feat,cur_semantic_seg = get_batch('test',index=0)#
        # ps_out collects the pre-normalized batch_data
        ps_out[i,...] = ps
        # ps_min is collected not from pc_normalize_min
        ps,ps_min = pc_normalize_min(ps)
        batch_data[i,...] = ps
        batch_label[i,:] = seg
        batch_smpw[i,:] = smpw
        batch_feats[i,:] = feat

        dropout_ratio = np.random.random()*0.875 # 0-0.875
        drop_idx = np.where(np.random.random((ps.shape[0]))<=dropout_ratio)[0]
        batch_data[i,drop_idx,:] = batch_data[i,0,:]
        batch_label[i,drop_idx] = batch_label[i,0]
        batch_smpw[i,drop_idx] *= 0
    	# ps_xyz_min collects the xyz minimums vector for each batch
        ps_xyz_min[i,...] = ps_min
    # ps_xyz_min and px_out are now returned.
    #cur_semantic_seg
    return batch_data, batch_label, batch_smpw, batch_feats, ps_xyz_min, ps_out,cur_semantic_seg

In [5]:
# train_f = open('Data/train_merge_min_norm_fea.pickle', 'rb')
# train_xyz, train_label, train_feats = pickle.load(train_f, encoding='bytes')
# train_xyz, train_label, train_feats = pickle.load(train_f)
# train_f.close()

#test_f = open('Data/test_merge_min_norm_fea_paper_height.pickle', 'rb')
#test_xyz, test_label, test_feats = pickle.load(test_f, encoding='bytes')
#test_feats = [tt[:,1:2] for tt in test_feats] #reflectance
#test_f.close()


##Store test sets under DATA folder
##Change input set here

testSet = np.loadtxt('Data/03labeled_seg.pts',skiprows=1)
#testSet = np.loadtxt('Data/CarOnly_small_mock.pts',skiprows=1)
#testSet = np.loadtxt('Data/Vaihingen3D_EVAL_WITHOUT_REF.pts',skiprows=1)
#testSet = np.loadtxt('Data/10.09Segment.pts',skiprows=1)

#testSet = np.loadtxt('Data/train_height.pts',skiprows=1)
label_w = testSet[:,4].astype('uint8')
if (testSet[:,3].max() != 0):
    testSet[:,3] = testSet[:,3]/testSet[:,3].max() #height above ground
if (testSet[:,5].max() != 0):
    testSet[:,5] = testSet[:,5]/testSet[:,5].max() #reflectance

# trainFeats = trainSet[:,[3,5]] #use reflectance and height above ground
testFeats = testSet[:,5:6] #only use reflectance

#ori 9
NUM_CLASSES = 9
label_values = range(NUM_CLASSES)

trainSet = np.loadtxt('Data/train_height.pts',skiprows=1)

label_w = trainSet[:,4].astype('uint8')
trainSet[:,3] = trainSet[:,3]/trainSet[:,3].max() #height above ground
trainSet[:,5] = trainSet[:,5]/trainSet[:,5].max() #reflectance

# trainFeats = trainSet[:,[3,5]] #use reflectance and height above ground
trainFeats = trainSet[:,5:6] #only use reflectance
#original 9
labelweights = np.zeros(9)
#ori 10
tmp,_ = np.histogram(label_w,range(10))
labelweights = tmp
labelweights = labelweights.astype(np.float32)
labelweights = labelweights/np.sum(labelweights)
labelweights = 1/np.log(1.4+labelweights)
print(labelweights)

#ori 9
labelweights_t = np.ones(9)
print(labelweights_t)

[2.9674518 2.0217078 1.9802364 2.9339767 2.874856  2.122891  2.7627096
 2.6275024 2.1882594]
[1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [6]:
def train_one_epoch(sess, ops, train_writer):
    """ ops: dict mapping from string to tf ops """
    is_training = True
    
    log_string('----')
    
   # Shuffle train samples
    train_idxs = np.arange(0, len(train_xyz))
    np.random.shuffle(train_idxs)
    num_batches = len(train_xyz)//BATCH_SIZE
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    
    for batch_idx in range(num_batches):
        
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        
        batch_data, batch_label, batch_smpw, batch_feats = get_batch_wdp('train', batch_idx)
        
        if batch_idx % (num_batches/2) == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))
        
        aug_data, rotation_matrix = provider.rotate_point_cloud_z(batch_data)
        
        feed_dict = {ops['pointclouds_pl']: aug_data,
                     ops['feature_pl']: batch_feats,
                     ops['labels_pl']: batch_label,
                     ops['smpws_pl']: batch_smpw,
                     ops['is_training_pl']: is_training,}
        summary, step, _, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred'], ops['learnrate']],
                                         feed_dict=feed_dict)
        train_writer.add_summary(summary, step)
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == batch_label)
        total_correct += correct
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
        
    log_string('learn rate: %f' % (lr_val))
    log_string('mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('accuracy: %f' % (total_correct / float(total_seen)))
    
    mloss = loss_sum / float(num_batches)
    macc = total_correct / float(total_seen)
    return mloss, macc

def eval_one_epoch_whole_scene(sess, ops):
    """ ops: dict mapping from string to tf ops """
    is_training = False
    
    total_correct = 0
    total_seen = 0
    loss_sum = 0
    total_seen_class = [0 for _ in range(NUM_CLASSES)]
    total_correct_class = [0 for _ in range(NUM_CLASSES)]
    
    log_string('----')
    
    test_idxs = np.arange(0, len(test_xyz))
    # Shuffle train samples
    np.random.shuffle(test_idxs)
    num_batches = len(test_xyz)//BATCH_SIZE
    
    #TEST_BATCH_SIZE = 1
    #num_batches = len(test_xyz)
    
    Confs = []
    
    #
    Confs_normalized = []
    pred_list = []
    
    is_continue_batch = False
    
    for batch_idx in range(num_batches):
        
        start_idx = batch_idx * BATCH_SIZE
        end_idx = (batch_idx+1) * BATCH_SIZE
        #cur_semantic_seg
        batch_data, batch_label, batch_smpw, batch_feats, ps_xyz_min, ps_out,cur_semantic_seg = get_batch_wdp_test('test', batch_idx)
        if batch_idx % (num_batches/2) == 0:
            print('Current batch/total batch num: %d/%d'%(batch_idx,num_batches))

            
        aug_data, rotation_matrix = provider.rotate_point_cloud_z(batch_data)
        
        feed_dict = {ops['pointclouds_pl']: aug_data,
                     ops['feature_pl']: batch_feats,
                     ops['labels_pl']: batch_label,
                     ops['smpws_pl']: batch_smpw,
                     ops['is_training_pl']: is_training}
        print("feed_dict: ",feed_dict)

        summary, step, _, loss_val, pred_val, lr_val = sess.run([ops['merged'], ops['step'], ops['train_op'], ops['loss'], ops['pred'], ops['learnrate']],
                                         feed_dict=feed_dict)
        
        pred_val = np.argmax(pred_val, 2)
        correct = np.sum(pred_val == batch_label)

        ##could as Michael for details
        ##pred_list.append([ps_out[0][0], pred_val])
        
        #pred_list.append([batch_data, pred_val])
        pred_list.append([ps_out, pred_val])
        ###print("pred_list: ",pred_list)
        
        total_correct += correct
        #total_seen += batch_data.shape[1]
        total_seen += (BATCH_SIZE*NUM_POINT)
        loss_sum += loss_val
        
       
        #NUM_POINT_fact = batch_data.shape[1]
        NUM_POINT_fact = (BATCH_SIZE*NUM_POINT)
        for i in range(BATCH_SIZE):
            for j in range(NUM_POINT):
                l = batch_label[i, j]
                total_seen_class[l] += 1
                total_correct_class[l] += (pred_val[i, j] == l)
                
        from sklearn.metrics import confusion_matrix
        Confs += [confusion_matrix(batch_label.flatten(), pred_val.flatten(), label_values)]
        #add normalize = true
        #x/total true viechles
        Confs_normalized += [confusion_matrix(batch_label.flatten(), pred_val.flatten(), label_values, normalize='true')]
    
    #end of for loop
    C = np.sum(np.stack(Confs), axis=0).astype(np.float32)
    C_normalized = np.sum(np.stack(Confs_normalized), axis=0).astype(np.float32)
    oa, avgF1 = Acc_from_confusions(C)
   
    log_string('learn rate: %f' % (lr_val))
    log_string('eval mean loss: %f' % (loss_sum / float(num_batches)))
    log_string('eval accuracy: %f'% (total_correct / float(total_seen)))
    log_string('eval avg class acc: %f' % (np.mean(np.array(total_correct_class)/np.array(total_seen_class,dtype=np.float))))
    
    mloss = loss_sum / float(num_batches)
    macc = total_correct / float(total_seen)
    #cur_semantic_seg,C
    return oa, avgF1,pred_list,cur_semantic_seg,Confs_normalized

In [7]:
print(tf.test.is_gpu_available())

True


In [8]:
###
test_xyz,test_label,test_feats = [],[],[]
num_test_samples = 1000
for i in range(num_test_samples):
    #cur_semantic_seg
    test_point_set, test_semantic_seg, test_sample_weight, test_feature_set,cur_semantic_seg = get_batch('test',0,
                                                                                            npoints=NUM_POINT)
    test_xyz.append(test_point_set)
    #test_label.append(test_semantic_seg)
    #test_weights.append(test_sample_weight)##
    #test_feats.append(test_feature_set)

In [9]:
print(tf.test.is_gpu_available())

True


In [10]:
# Just clears out previous sessions
tf.reset_default_graph()
K.clear_session()

# Run the session, add the parameters to the graph, testing is run in the eval_one_epoch_whole_scene line
with tf.Session() as sess:
    pointclouds_pl, labels_pl, smpws_pl = placeholder_inputs(BATCH_SIZE, NUM_POINT)
    feature_pl = tf.placeholder(tf.float32, shape=(None, None, 1))
    is_training_pl = tf.placeholder(tf.bool, shape=())

    # Note the global_step=batch parameter to minimize. 
    # That tells the optimizer to helpfully increment the 'batch' parameter for you every time it trains.
    batch = tf.Variable(0)
    bn_decay = get_bn_decay(batch)
    tf.summary.scalar('bn_decay', bn_decay)

    # Get model and loss 
    pred, end_points = MODEL.get_model(pointclouds_pl, is_training_pl, NUM_CLASSES, bn_decay=bn_decay, feature=feature_pl)
    loss = MODEL.get_loss(pred, labels_pl, smpws_pl)

    tf.summary.scalar('loss', loss)

    correct = tf.equal(tf.argmax(pred, 2), tf.to_int64(labels_pl))
    accuracy = tf.reduce_sum(tf.cast(correct, tf.float32)) / float(BATCH_SIZE*NUM_POINT)
    tf.summary.scalar('accuracy', accuracy)

    # Get training operator
    learning_rate = get_learning_rate(batch)
    tf.summary.scalar('learning_rate', learning_rate)
    if OPTIMIZER == 'momentum':
        optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=MOMENTUM)
    elif OPTIMIZER == 'adam':
        optimizer = tf.train.AdamOptimizer(learning_rate)
    train_op = optimizer.minimize(loss, global_step=batch)
    
    # Add summary writers
    merged = tf.summary.merge_all()
    
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    ops = {'pointclouds_pl': pointclouds_pl,
       'labels_pl': labels_pl,
       'feature_pl': feature_pl,
       'smpws_pl': smpws_pl,
       'is_training_pl': is_training_pl,
       'pred': pred,
       'loss': loss,
       'train_op': train_op,
       'merged': merged,
       'step': batch,
       'learnrate': learning_rate}
    
    saver = tf.train.import_meta_graph(os.path.join(LOG_DIR, 'model.ckpt.meta'))
    saver.restore(sess, tf.train.latest_checkpoint(LOG_DIR))

    # This runs the testing
    #cur_semantic_seg,C
    test_acc, test_F1, pred_list,cur_semantic_seg,C_n = eval_one_epoch_whole_scene(sess, ops)
    print("pred_list: ",pred_list)

# Not sure we need this, but clears the session
K.clear_session()


INFO:tensorflow:Restoring parameters from log_wen_v16_sample8192_group2_lw14_F1_noheight/model.ckpt
----
Current batch/total batch num: 0/166
feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 8.6294380e+06, -1.1061392e+07,  1.8879533e+08],
        [ 8.6294380e+06, -1.1061392e+07,  1.8879533e+08],
        [ 8.6294380e+06, -1.1061392e+07,  1.8879533e+08],
        ...,
        [ 8.6294380e+06, -1.1061392e+07,  1.8879533e+08],
        [ 2.8723278e+07,  3.2788394e+05,  2.1342085e+08],
        [ 8.6294380e+06, -1.1061392e+07,  1.8879533e+08]],

       [[ 1.8647878e+07,  2.2065816e+07,  1.0023408e+05],
        [ 1.8647878e+07,  2.2065816e+07,  1.0023408e+05],
        [ 1.8647878e+07,  2.2065816e+07,  1.0023408e+05],
        ...,
        [ 1.8647878e+07,  2.2065816e+07,  1.0023408e+05],
        [ 1.8647878e+07,  2.2065816e+07,  1.0023408e+05],
        [ 1.8431008e+07,  2.2147184e+07,  9.8822008e+04]],

       [[ 5.9934220e+06, -2.1862792e+07,  4.7436004e+07],
  

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 1.3121723e+07,  7.1231105e+06,  2.5213742e+06],
        [ 1.3121723e+07,  7.1231105e+06,  2.5213742e+06],
        [ 8.3191680e+06,  4.5868955e+06,  2.2446728e+06],
        ...,
        [ 1.9039134e+07,  2.6055724e+07,  3.8775862e+05],
        [ 1.3121723e+07,  7.1231105e+06,  2.5213742e+06],
        [ 1.3121723e+07,  7.1231105e+06,  2.5213742e+06]],

       [[ 1.1747364e+05,  1.3399024e+07,  8.9617490e+06],
        [ 6.4343925e+06,  2.3984114e+07,  9.0191600e+06],
        [ 7.8326140e+06,  4.3293125e+06,  2.3980820e+06],
        ...,
        [ 1.9581924e+07,  2.5925706e+07,  9.0784540e+06],
        [ 6.5397605e+06,  9.8686120e+06,  9.0379840e+06],
        [ 2.2773390e+07,  1.3819091e+07,  1.2846845e+05]],

       [[-6.9917365e+06,  1.5885792e+07,  1.9998574e+08],
        [-6.9917365e+06,  1.5885792e+07,  1.9998574e+08],
        [-1.0190279e+07,  2.4453902e+07,  2.0006904e+08],
        ...,
        [-6.991

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[  5560153.  ,  29322728.  ,    330818.34],
        [  5560153.  ,  29322728.  ,    330818.34],
        [  5560153.  ,  29322728.  ,    330818.34],
        ...,
        [  5560153.  ,  29322728.  ,    330818.34],
        [  5560153.  ,  29322728.  ,    330818.34],
        [  5560153.  ,  29322728.  ,    330818.34]],

       [[-29249620.  ,   6460720.  ,  45730620.  ],
        [-20628356.  ,  26020496.  ,  45842612.  ],
        [ -7999211.  ,  13649666.  ,  45815790.  ],
        ...,
        [-29249620.  ,   6460720.  ,  45730620.  ],
        [-29086302.  ,  11028886.  ,  45741910.  ],
        [-17826474.  ,  17669148.  ,  45764028.  ]],

       [[-15951778.  , -30805466.  ,    333171.28],
        [-20508854.  , -19617626.  ,   2397140.8 ],
        [-15951778.  , -30805466.  ,    333171.28],
        ...,
        [-15951778.  , -30805466.  ,    333171.28],
        [ -7568110.5 , -23838174.  ,   4440875.5 ],
 

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[-1.4855783e+07, -1.6810668e+07,  1.9529106e+05],
        [-2.2728646e+07, -5.2827117e+04,  7.6704969e+04],
        [-1.4855783e+07, -1.6810668e+07,  1.9529106e+05],
        ...,
        [-1.4855783e+07, -1.6810668e+07,  1.9529106e+05],
        [-1.4855783e+07, -1.6810668e+07,  1.9529106e+05],
        [-1.4855783e+07, -1.6810668e+07,  1.9529106e+05]],

       [[-1.3931848e+07,  2.3595366e+07,  1.5293873e+05],
        [-1.3931848e+07,  2.3595366e+07,  1.5293873e+05],
        [ 2.8346105e+06,  3.0163660e+07,  2.2776136e+05],
        ...,
        [-1.3931848e+07,  2.3595366e+07,  1.5293873e+05],
        [-7.6186112e+05,  6.3732570e+06,  1.3505669e+05],
        [-1.2497798e+07,  2.8025066e+07,  1.4211552e+05]],

       [[-1.4597973e+07,  2.8930890e+07,  2.7340781e+05],
        [-2.0706564e+07,  2.6837818e+07,  2.5176105e+05],
        [-1.5471965e+07,  7.3520115e+06,  1.2705699e+05],
        ...,
        [-3.663

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[-5.67841950e+06, -3.65293480e+07,  1.05880828e+05],
        [-5.67841950e+06, -3.65293480e+07,  1.05880828e+05],
        [-5.67841950e+06, -3.65293480e+07,  1.05880828e+05],
        ...,
        [-5.67841950e+06, -3.65293480e+07,  1.05880828e+05],
        [-5.67841950e+06, -3.65293480e+07,  1.05880828e+05],
        [-5.67841950e+06, -3.65293480e+07,  1.05880828e+05]],

       [[ 2.29325260e+07, -1.54500330e+07,  1.33645234e+05],
        [ 1.40155150e+07, -2.09131300e+07,  1.24704055e+05],
        [ 1.68090400e+07, -2.39507680e+07,  1.12469062e+05],
        ...,
        [ 2.73296740e+07, -1.61053810e+07,  1.48233156e+05],
        [ 5.92962350e+06, -3.52230475e+06,  1.73644625e+05],
        [ 2.12507860e+07, -5.77282109e+04,  1.23292289e+05]],

       [[ 2.78242540e+07,  1.55744138e+06,  3.43523320e+04],
        [ 2.78242540e+07,  1.55744138e+06,  3.43523320e+04],
        [ 2.78242540e+07,  1.55744138e+06,  

...

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[-2.4681590e+07, -1.7239032e+07,  2.4423164e+05],
        [-2.4643836e+07, -1.7922638e+07,  2.3576136e+05],
        [-2.0707458e+07, -1.1771933e+07,  1.9576166e+05],
        ...,
        [-2.4681590e+07, -1.7239032e+07,  2.4423164e+05],
        [-2.4681590e+07, -1.7239032e+07,  2.4423164e+05],
        [-1.0159902e+07, -1.8687580e+07,  2.0235020e+05]],

       [[-3.3088190e+07, -1.3761208e+07,  3.0823041e+05],
        [-3.3088190e+07, -1.3761208e+07,  3.0823041e+05],
        [-3.3088190e+07, -1.3761208e+07,  3.0823041e+05],
        ...,
        [-1.5163812e+07, -1.6608682e+07,  7.3410547e+04],
        [-3.3088190e+07, -1.3761208e+07,  3.0823041e+05],
        [-2.6658884e+07,  6.1424825e+06,  3.1999481e+05]],

       [[ 1.5492151e+07, -3.2437920e+06,  3.9858244e+05],
        [ 4.6957140e+06, -2.5571500e+07,  3.5811247e+05],
        [ 1.5492151e+07, -3.2437920e+06,  3.9858244e+05],
        ...,
        [ 1.549

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[  9163096.   ,   9816785.   ,    189644.02 ],
        [  9163096.   ,   9816785.   ,    189644.02 ],
        [ 13205961.   ,  29437204.   ,    111998.17 ],
        ...,
        [  9163096.   ,   9816785.   ,    189644.02 ],
        [ -2353361.8  ,  29828072.   ,     64939.957],
        [  6843249.   ,  33525266.   ,     84704.664]],

       [[-12101852.   ,   3700052.   ,  47718820.   ],
        [-16371049.   , -23841744.   ,  47582824.   ],
        [-12101852.   ,   3700052.   ,  47718820.   ],
        ...,
        [-23701196.   ,  -9878340.   ,  47687764.   ],
        [-18889412.   ,   3745575.   ,  47716468.   ],
        [-32009342.   ,  -8136966.   ,  47700468.   ]],

       [[  8438307.   , -21984252.   ,   6806017.   ],
        [  8438307.   , -21984252.   ,   6806017.   ],
        [ -9613400.   , -25721456.   ,   6837075.   ],
        ...,
        [  8438307.   , -21984252.   ,   6806017.   ],
     

feed_dict:  {<tf.Tensor 'Placeholder:0' shape=(?, ?, 3) dtype=float32>: array([[[ 1.20386550e+07,  1.58236170e+07,  1.59527281e+05],
        [ 1.20386550e+07,  1.58236170e+07,  1.59527281e+05],
        [ 1.20386550e+07,  1.58236170e+07,  1.59527281e+05],
        ...,
        [ 1.20386550e+07,  1.58236170e+07,  1.59527281e+05],
        [ 1.20386550e+07,  1.58236170e+07,  1.59527281e+05],
        [ 1.20386550e+07,  1.58236170e+07,  1.59527281e+05]],

       [[-9.28770200e+06, -1.80928060e+07,  6.59425520e+07],
        [-9.28770200e+06, -1.80928060e+07,  6.59425520e+07],
        [-1.37438420e+07, -2.55498200e+07,  6.59218480e+07],
        ...,
        [-9.28770200e+06, -1.80928060e+07,  6.59425520e+07],
        [-9.48183438e+04, -3.51586920e+07,  6.58522000e+07],
        [-7.50218500e+06, -2.92411460e+07,  6.58870240e+07]],

       [[-2.54186960e+07, -1.40126290e+07,  7.29402656e+04],
        [-1.80524875e+06, -2.27036525e+06,  1.09174945e+05],
        [-2.54186960e+07, -1.40126290e+07,  

Overall accuracy： 0.00  Average F1 score： 0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 
 0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 

learn rate: 0.000020
eval mean loss: 11.612732
eval accuracy: 0.000000
eval avg class acc: nan
pred_list:  [[array([[[1.04150876e+08, 2.05305042e+08, 2.64973949e+08],
        [1.07910987e+08, 2.15325580e+08, 2.89739229e+08],
        [1.04120338e+08, 2.05628330e+08, 2.77757291e+08],
        ...,
        [1.05627820e+08, 2.08805502e+08, 2.92494483e+08],
        [9.55097187e+07, 2.26724855e+08, 2.89599466e+08],
        [1.09312215e+08, 2.09774008e+08, 2.49819821e+08]],

       [[2.44586442e+08, 1.34760514e+08, 2.42017584e+08],
        [2.41056169e+08, 1.34376084e+08, 2.42022760e+08],
        [2.40588662e+08, 1.39256638e+08, 2.41993114e+08],
        ...,
        [2.39180761e+08, 1.53501649e+08, 2.42022760e+08],
        [2.26525788e+08, 1.40977673e+08, 2.41953114e+08],
        [2.

/home/az/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:132: RuntimeWarning: invalid value encountered in true_divide


In [13]:
cur_semantic_seg

array([2, 2, 2, ..., 5, 5, 5], dtype=int32)

In [12]:
len(cur_semantic_seg)
#9371

9710

In [14]:
print(test_acc)
print(test_F1)

0.13409351
0.03830105


In [12]:

pt = np.empty([len(pred_list),len(pred_list[0][0]),len(pred_list[0][0][0]),4])
for batch in range(0,len(pred_list)):
    for i in range(0,len(pred_list[batch][0])):
        pt[batch][i]=np.column_stack((pred_list[batch][0][i],pred_list[batch][1][i]))

In [13]:
#//X Y Z Classification
#change output path here, FULL PATH
f = open("/home/az/Desktop/labeled_1000000x.txt", "w")
f.write("//X Y Z Classification\n")

23

In [14]:
##print out results
for l1 in pt:
    for l2 in l1:
        for l3 in l2:
            #s = ", ".join(map(str, l3))
            s = " ".join(map(str, l3))
            for item in s:
                f.write(item)
            f.write('\n')
f.close()

In [11]:
C

array([[0.000000e+00, 4.579000e+03, 2.900000e+01, 2.000000e+00,
        1.607000e+03, 9.149700e+04, 1.703708e+06, 1.635600e+04,
        1.597370e+05],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 7.000000e+00, 0.000000e+00,
        2.800000e+01, 9.241000e+03, 4.679700e+04, 8.990000e+02,
        5.321000e+03],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000

In [13]:
array([[0.000000e+00, 4.579000e+03, 2.900000e+01, 2.000000e+00,1.607000e+03, 9.149700e+04, 1.703708e+06, 1.635600e+04,1.597370e+05],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00],
       [0.000000e+00, 0.000000e+00, 7.000000e+00, 0.000000e+00,2.800000e+01, 9.241000e+03, 4.679700e+04, 8.990000e+02,5.321000e+03],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
       [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,0.000000e+00]], dtype=float32)

In [ ]:
#sum 3 = 